# Stokes coefficients estimation
In this set of notebooks, we show simulations of retrival of a set of real spherical harmonics gravity coefficients from a set of gravitational measurements (either potential or acceleration). This is a simplified example of what is performed in an orbit determination (OD) campaign, where the Stokes coefficients are estimated, along with other parameters, from a set of radio-tracking measurements of an orbiting spacecraft.

## Gravity and spherical harmonics
The gravitational potential U at a point $(r, \theta, \phi)$ - radius, colatitude, and longitude - is modelled as a series of spherical harmonics truncated at degree `n_max`:
$$U(r,\theta,\phi) = \frac{\mu}{r}\sum_{n=0}^{n_{\max}} \sum_{m=0}^{m=l}  {\left(\frac{R_0}{r}\right)}^n P_{lm}(\cos{\theta})\left[C_{lm}cos{m\lambda}+S_{lm}\sin{m\lambda}\right]$$
where $\mu$ (in the code, `gm`) is the product of the mass of the central body and the universal gravitational constant, and $R_0$ a reference radius. The spherical harmonics functions and corresponding Stokes coeffcients ($C_{lm}, S_{lm}$), are real-valued, $4\pi$-normalized, and exclude the Condon-Shortley phase factor of $(-1)^m$. More details can be found in the relevant SHTOOLS [documentation page](https://shtools.github.io/SHTOOLS/real-spherical-harmonics.html). They correspond indeed to the default convention for real spherical harmonics in SHTOOLS.
We follow the geodesy and geophysics sign conventions, with the potential positive and $\mathbf{g} = +\nabla U $ .  
The gravitational accelerations is then given by the gradient of the potential, that is:
$$\mathbf{g} = \nabla U = \frac{\partial U}{\partial r} \mathbf{u_r} + \frac{1}{r}\frac{\partial U}{\partial \theta} \mathbf{u_\theta}+ \frac{1}{r\sin{\theta}} \frac{\partial U}{\partial \lambda} \mathbf{u_\lambda}$$
Therefore, the three components in the directions of $\mathbf{u_r}, \mathbf{u_\theta}$, and $\mathbf{u_\phi}$ are:
$$g_r = -\frac{\mu}{r^2}\sum_{n=0}^{n_{\max}} \sum_{m=0}^{m=l} (n+1) {\left(\frac{R_0}{r}\right)}^n P_{lm}(\cos{\theta})\left[C_{lm}\cos{m\lambda}+S_{lm}\sin{m\lambda}\right]$$
$$g_\theta = \frac{\mu}{r^2}\sum_{n=0}^{n_{\max}} \sum_{m=0}^{m=l} {\left(\frac{R_0}{r}\right)}^n \frac{\partial P_{lm}(\cos{\theta})}
{\partial \theta}\left[C_{lm}\cos{m\lambda}+S_{lm}\sin{m\lambda}\right]$$
$$g_\lambda = \frac{\mu}{r^2}\sum_{n=0}^{n_{\max}} \sum_{m=0}^{m=l} {\left(\frac{R_0}{r}\right)}^n m \frac{P_{lm}(\cos{\theta})}{\sin{\theta}}\left[-C_{lm}\sin{m\lambda}+S_{lm}\cos{m\lambda}\right]$$

## Problem formulation
We strive to reconstruct a the gravitational field of a body, in the form of its Stokes coefficients $C_{lm}, S_{lm}$, starting from measurements of $U$ or $\nabla U$ obtained over discrete points in $\mathbb{R}^3$.

## Limitations
There are some critical differences between what is shown here and what is actually done in orbit determination.
- In OD, **gravity measurements are not directly available**, since the tracked object is in free-fall. Where present, an accelerometer can provide measurements of all non-gravitational effects, which can then be removed from the total acceleration predicted for the spacecraft. In general, however, an initial dynamical model is constructed, which predicts the effect of both gravitational and non-gravitational forces acting on the spacecraft. Starting from an *a priori* initial state for the spacecraft, its trajectory is numerically propagated according to this dynamical model, and a measurement model is used to generate predicted values for the observables at the time-stamps of the available measurements. Then a filter is used to minimize the discrepancy between computed and raw measurements, by estimating (through iterations) corrections for the parameters of the dynamical model and for the initial state of the propagated trajectory.
- In OD, the available measurements are typically **1-dimensional**. Mainly, Doppler and range measurements are used, which provide, respectively, the relative velocity and the distance between a spacecraft and a ground station on Earth, along the line-of-sight. Here, instead, we assume that gravity measurements are available directly, at specific points along the trajectory. This presupposes a preliminar orbit reconstruction step, where the gravity information is extracted from the radio-tracking data. As the accuracy of the gravity reconstruction can be assumed to scale with the coverage of the radio-tracking measurements, we still base the selection of the gravity measurements points on realistic tracking schedules for Doppler data.
- Our OD software (MONTE) relies on an **Extended Kalman Filter** for the parameter estimation. Here, as there are no time-variable parameters, we opt for a simple batch least-squares inversion instead.

## **Case 1**: Fitting gravity acceleration points over a regular grid
In this very ideal case, we assume that the gravity information is available on a spherical grid. The grid points are those of the Gauss-Legendre Quadrature grid in SHTOOLS, which therefore depend on the maximum degree of the spherical harmonics coefficients

Dependencies can be installed via pip (using the requirements.txt file). We use plotly for graphics, pyshtools for handling spherical harmonics, and spiceypy for orbital computations

In [1]:
import os
from time import time

import numpy as np
import plotly.graph_objects as go
import pyshtools as sh
import matplotlib.pyplot as plt
import pickle as pk
import json

import scripts
from scripts._units import *

from IPython.display import IFrame
import plotly.io as pio
pio.templates.default = "plotly_white"

### Settings

In [2]:
out_name = "glq_grid" # outputs destination folder (a subfolder of out)
out_path = os.path.join("out", out_name)
if not os.path.exists(out_path):
    os.makedirs(out_path)

use_potential = False # Whether to work with potential (flag to True) or with accelerations (flag to False)
perturb_msr = False # Whether to introduce additive Gaussian noise in the syntehtic measurements
drop_traj_points = False # Only keep locations of measurement points, to save on memory (needed when using realistic trajectories) 
compute_covariance = True # Additionally compute uncertainties for the estimated coefficients (~20% increase in inversion time)
check_acc_msr = True # Compare acceleartion measurements with those given by pyshtools

msr_dim = 1 if use_potential else 3 # dimensions of each measurerment (i.e. scalar or vector)
partials_func = scripts.pot_cnm_partials if use_potential else scripts.acc_cnm_partials # function to use for partials computation
val_lbl = "U_grav (m^2/s^2)" if use_potential else "|a_grav| (m/s^2)" # label for plots
rng = np.random.default_rng()

### Ground-truth gravity
For this case, we select as ground-truth the gravity coefficients of Mars, as estimated by Konopliv, et al. (2016), and available on the [NASA PDS](https://pds-geosciences.wustl.edu/mro/mro-m-rss-5-sdp-v1/mrors_1xxx/data/shadr/). The coefficients and their uncertainties are provided up to degree and order 120. This solution, MRO120D, was obtained by processing radio-tracking data of 3 Martian orbiters (and 3 landers), acquired over a span of 15 years, for a total of about 5 million measurement points (when converted to a 60s count time). Replicating such an extended tracking schedule would be excessively computationally intensive. Here, we limit our estimation to below 100 thousand points. We expect, however, that given our strong simplification of the OD process (as described above), a significantly smaller amount of observations compared to real OD campaign is required to reach comparable gravity reconstruction accuracies.

In this case, we deal with 2 sets of coefficients: one used to generate synthetic measurements (`_sim`) and whose values are read from MRO120D, and the other (`_est`) whose values have to be estimated. The two sets of coefficients can have a different maximum degree.

For compatibility with SHTOOLS, we store the coefficients in matrices of size `(2,n_max+1,n_max+1)`. However, for computations we rely on 1D arrays. The array `cnm_idx` associates to each index of the vector a tuple of indices for the matrix, and vice-versa for the dictionary `cnm_map`.

In [3]:
# SH degree cutoff (for both simulation and estimation, and in any case limited by that of the file)
n_max_sim = 120 # macimum degree of the coefficients used to simulate the measurements
n_max_est = 100 # maximum degree of the coefficients to be estimated
n_max = max(n_max_sim, n_max_est)

# Read SH coefficients, and cut to n_max_sim
file_name = "spice/data/jgmro_120d_sha.tab"
cnm_mat, r_0, gm = scripts.read_SHADR(file_name)
cnm_mat_sim = cnm_mat[:, : n_max_sim + 1, : n_max_sim + 1]

# Index array and dictionary to go from a matrix of Cnm to a 1D array, and vice-versa
cnm_idx_sim, cnm_map_sim = scripts.get_cnm_idx(n_max_sim)
cnm_idx_est, cnm_map_est = scripts.get_cnm_idx(n_max_est)

### Points selection
In this simple case, there is no trajectory behind our selection of the measurement points, which are instead sampled over the SHTOOLS GLQ grid for degree `n_max_sim` and at a constant altitude of 234 km, close to the altitude of the Mars Reconnaissance Orbiter (MRO) spacecraft.

In [4]:
# Get cartesian and spherical coordinates of the grid points
state_mat, sph_coords = scripts.points_from_grid(n_max_sim, r=(r_0 + 234.0 * km)) 
n_msr_pts = state_mat.shape[0]

## Forward model

### Simulating measurements
Given the measurements locations and the matrix of Stokes coefficients, the gravity measurements can be simulated following the equations above. If the simulation and estimation sets of coefficients have the same maximum degree, however, the computation of the synthetic measurements is performed in the next cell, at the same time as the computation of the partials.


In [5]:
sim_msr_vec = None
if n_max_sim != n_max_est:
    print("Computing synthetic measurements...")
    ts = time()

    # Compute potential and acceleration at the selected coordinates
    pot_sim, acc_sim = scripts.compute_pot_acc(
        cnm_mat_sim, r_0, sph_coords
    )
    pot_sim *= gm
    acc_sim *= gm
    # Pick right observable, and convert to 1D array
    if use_potential:
        sim_msr_vec = pot_sim.reshape(n_msr_pts * msr_dim)
    else:
        sim_msr_vec = acc_sim.reshape(n_msr_pts * msr_dim)

    # A little hacky, but for now this is stored as a bool to economize on memory
    cnm_mat_est = np.empty((2, n_max_est + 1, n_max_est + 1), dtype=bool)

    # Compare the obtained acceleration values with those computed by SHTOOLS
    if check_acc_msr:
        sh_coeffs = sh.SHGravCoeffs.from_array(cnm_mat_sim, gm, r_0)
        acc_shtools = np.array(
            [
                sh_coeffs.expand(colat=acc[1] / deg, lon=acc[2] / deg, a=acc[0])
                for acc in sph_coords
            ]
        )
        print(
            "Acceleration error wrt SHTOOLS",
            np.linalg.norm(acc_sim - acc_shtools, axis=0),
        )
    print("Took {:.2f} s".format(time() - ts))
else:
    # I the maximum degrees are the same, the computation is done below
    cnm_mat_est = cnm_mat_sim.copy()

Computing synthetic measurements...


100%|███████████████████████████████████| 121/121 [00:30<00:00,  3.92it/s]


Acceleration error wrt SHTOOLS [5.19261646e-14 7.23254065e-16 6.86207023e-16]
Took 41.39 s


### Computing partials
The relation between gravity and the Stokes coefficient is linear, and can be expressed via the system:
$$\mathbf{z} = \mathbf{H}\mathbf{x} + \mathbf{\nu}$$
If $\mathbf{\tilde{H}}$ is the matrix of the partials w/r/t the Stokes coefficients and $\mathbf{\tilde{z}}$ the vector of measurements with covariance $\mathbf{W}$, then
$$\mathbf{H} = \mathbf{W}^{1/2}\tilde{\mathbf{H}},$$ 
$$\mathbf{z} = \mathbf{W}^{1/2}\tilde{\mathbf{z}},$$
so as to have the weighted errors $\mathbf{\nu}$ be independent and with unitary variance, meaning $E[\mathbf{\nu}] = \mathbf{I}$. The expression of the partials of the measurements with respect to the Stokes coefficients can be easily derived from the formulas in the first cell. Here we assume that $\mathbf{W}$ is diagonal, i.e. that the measurements are independent, each with noise of standard deviation $\sigma_i$, stored in the vector (or scalar) `msr_noise` in the code.

The least-squares solution $\hat{\mathbf{x}}$ of this over-determined system can be found by inverting the normal equations:
$$\mathbf{H}^T\mathbf{H}\hat{\mathbf{x}} = \mathbf{H}^T\mathbf{z}$$
where we define $\mathbf{N}=\mathbf{H}^T\mathbf{H}$ (the normal matrix) and $\mathbf{y} = \mathbf{H}^T\mathbf{z}$. 
#### Using normal equations  
As suggested by Jorge, in order to limit the memory requirements, we form the normal equations directly, so that we only need to store a matrix of size `(n_params, n_params)`, instead of the partials matrix of size `(n_msr, n_params)`. It is of course not advisable to go through the normal equations when solving a least squares system, but instead perform a decomposition of the matrix $\mathbf{H}$, because of the higher possibility for numerical errors (the condition number of $\mathbf{N}$ is the square of that of $\mathbf{H}$.  However, since operating on $\mathbf{H}$ is deemed too computationally expensive for `n_max_est`>50 and the system appears to be generally well-conditioned, we opt for the normal-equations approach. A low-memory-usage alternative would be to use a sequential filter, but our Python implementation of a Kalman filter was too slow to be applied in realistic estimation scenarios.

$\mathbf{N}$ and $\mathbf{y}$ are populated via batches of `batch_size` measurements, where `batch_size` can be increased in order to speed-up the computation, at the cost of increased memory usage. The contributions of each batch to both $\mathbf{N}$ and $\mathbf{y}$ can be simply added to form the final normal equations.

In [6]:
print("Computing Normal equations...")
ts = time()

# measurements standard deviation. Can be a vector of size n_msr or a float. Here we take 1e-2% of the surface gravity.
msr_noise = 1e-4 * gm / np.power(r_0, 1 if use_potential else 2)
# in the scripts, the partials are computed up to a factor of gm
partials_scale = gm

# Computing the terms of the normal equations
# If sim_msr_vec is not None, it will be copied to msr_vec (and used to compute y),
# otherwise msr_vec and y are computed from the Stokes coefficients
N_mat, y, msr_vec = scripts.compute_normal_equations(
    cnm_mat_est, # coefficients used to simulate measurements (if sim_msr_vec is None)
    sph_coords, # measurements locations 
    partials_func, # function for the partials computation
    r_0=r_0,
    batch_size=1000, # With 1000, about 2GB RAM for n_max_est=100 and 5e4 msr
    msr_noise=msr_noise,
    partials_scale=partials_scale,
    raw_msr_vec=sim_msr_vec, # synthetic measurements (if None they will be computed from cnm_mat_est)
    perturb_msr=perturb_msr,  # contaminate synthetic measurements with Gaussian noise
    rng=rng,
)
print("Took {:.2f} s".format(time() - ts))

Computing Normal equations...


100%|█████████████████████████████████████| 30/30 [02:48<00:00,  5.62s/it]

Took 168.94 s


### Least-squares solution
The least-squares solution for the vector of Stokes coefficients is given by
$$\hat{\mathbf{x}} = (\mathbf{H}^T\mathbf{H})^{-1}\mathbf{H}^T\mathbf{z}$$
If no covariance of the solution is required, the system of normal equations is solved via `numpy.linalg.lstsq`.
Otherwise, we compute the singular-value decomposition of $\mathbf{N}$, so that $\mathbf{N} = \mathbf{U} \mathbf{S} \mathbf{V^T}$, with $\mathbf{U}$ and $\mathbf{V}$ orthonormal matrices ($\mathbf{V}\mathbf{V^T} = \mathbf{I}$), and $\mathbf{S}$ the diagonal matrix of the singular values of $\mathbf{N}$.

The covariance of the estimated parameters will then be
$$\mathbf{P} = \mathbf{V}\mathbf{S^{-1}} \mathbf{U^T}$$
and the solution
$$\hat{\mathbf{x}} = \mathbf{P}\mathbf{y}$$ 

In [7]:
cnm_vec_sim = cnm_mat_sim[*cnm_idx_sim.T]  # ground-truth vector of Stokes coefficients

# solution and covariance for the estimated parameters
# if compute_covariance is False, cov will be None 
cnm_vec_est, cov = scripts.solve_normal_equations(
    N_mat, y, compute_covariance=compute_covariance
)

if compute_covariance:
    cnm_vec_sigma = np.sqrt(np.diagonal(cov))  # formal errors
    corr_mat = cov / np.outer(cnm_vec_sigma, cnm_vec_sigma)  # correlations matrix

# difference between estimated and ground-truth Stokes coefficients
n_shared_coeffs = min(cnm_vec_sim.shape[0], cnm_vec_est.shape[0])
cnm_true_errors = cnm_vec_est[:n_shared_coeffs] - cnm_vec_sim[:n_shared_coeffs]


Solving via SVD...
Took 777.02 s


### Plotting

In [8]:
val = msr_vec.reshape(-1, msr_dim)

# 3D trajectory with measurement points
fig_trj = scripts.plot_traj(state_mat, val, val_lbl=val_lbl)
fig_out_path = os.path.join(out_path, "sim_msr_plot_3D.html")
fig_trj.write_html(
            fig_out_path,
            include_mathjax="cdn",
            include_plotlyjs="cdn",
            config=dict({"scrollZoom": True}),
        )
IFrame(src=fig_out_path, width=1000, height=500)

In [15]:
# 2D plot of trajectory and measurements (in spherical coordinates)
plot_coords  = sph_coords[:, [2, 1]]
plot_coords[:,1] = np.pi/2-plot_coords[:,1]
fig_grav_2d = scripts.plot_val_2d(
    plot_coords/deg, val, val_lbl=val_lbl
)
fig_out_path = os.path.join(out_path, "sim_msr_plot_2D.html")
fig_grav_2d.write_html(
            fig_out_path,
            include_mathjax="cdn",
            include_plotlyjs="cdn",
            config=dict({"scrollZoom": True}),
        )
IFrame(src=fig_out_path, width=1000, height=500)

In [10]:
# Spectra of Sokes coefficients and their errors
cnm_spectrum_sim = np.array(
    [
        (n, np.linalg.norm(cnm_vec_sim[cnm_idx_sim[:, 1] == n]) / np.sqrt(2 * n + 1))
        for n in range(n_max_sim + 1)
    ]
)
cnm_spectrum_est = np.array(
    [
        (n, np.linalg.norm(cnm_vec_est[cnm_idx_est[:, 1] == n]) / np.sqrt(2 * n + 1))
        for n in range(n_max_est + 1)
    ]
)

# Line plots of the Cnm spectra
fig_spectrum = go.Figure()
fig_spectrum.add_traces(
    go.Scatter(
        x=cnm_spectrum_sim[2:, 0],
        y=cnm_spectrum_sim[2:, 1],
        mode="lines",
        line=dict(color="darkgrey", width=2),
        showlegend=True,
        name="Ground-truth",
    )
)
fig_spectrum.add_traces(
    go.Scatter(
        x=cnm_spectrum_est[2:, 0],
        y=cnm_spectrum_est[2:, 1],
        mode="lines",
        line=dict(color="royalblue", width=2),
        showlegend=True,
        name="Estimated",
    )
)
if compute_covariance:
    cnm_spectrum_sigma = np.array(
        [
            (
                n,
                np.linalg.norm(cnm_vec_sigma[cnm_idx_est[:, 1] == n])
                / np.sqrt(2 * n + 1),
            )
            for n in range(n_max_est + 1)
        ]
    )
    fig_spectrum.add_traces(
        go.Scatter(
            x=cnm_spectrum_sigma[2:, 0],
            y=cnm_spectrum_sigma[2:, 1],
            mode="lines",
            line=dict(color="royalblue", dash="dash", width=2),
            showlegend=True,
            name="Uncertainty",
        )
    )
fig_spectrum.update_layout(
    xaxis_title=r"$l_{\max}$", yaxis_title="Power spectrum", yaxis_type="log"
)
fig_out_path = os.path.join(out_path, "spectrum.html")
fig_spectrum.write_html(
            fig_out_path,
            include_mathjax="cdn",
            include_plotlyjs="cdn",
            config=dict({"scrollZoom": True}),
        )
IFrame(src=fig_out_path, width=800, height=500)

#### Error statistics
Here we evaluate the synthetic and the estimated Stokes coefficients over a same GLQ grid, computed for degree `n_max` (maximum of `n_max_sim` and `n_max_est`) and at a radial distance of $1.1R_0$

In [11]:
print("Computing errors on GLQ grid...")
ts = time()

# constructing matrix from vector of estimated coefficients
cnm_mat_est = cnm_mat_est.astype(np.double)
cnm_mat_est[*cnm_idx_est.T] = cnm_vec_est

# cartesian (state_grid) and spherical(rtp_grid) coordinates of the GLQ grid points
state_grid, rtp_grid = scripts.points_from_grid(n_max, r=(r_0 * 1.1), use_GLQ_grid=True)

# forward gravity computation for ground-truth coefficients
# The function outputs are up to a factor of gm, so we multiply them by gm
pot_sim_grid, acc_sim_grid = [
    el * gm for el in scripts.compute_pot_acc(cnm_mat_sim, r_0, rtp_grid)
]
# forward gravity computation for estimated coefficients
pot_est_grid, acc_est_grid = [
    el * gm for el in scripts.compute_pot_acc(cnm_mat_est, r_0, rtp_grid)
]
print("Took {:.2f} s".format(time() - ts))

Computing errors on GLQ grid...


100%|███████████████████████████████████| 101/101 [00:30<00:00,  3.27it/s]

Took 71.32 s


In [12]:
# difference
err_grid = pot_sim_grid - pot_est_grid if use_potential else acc_sim_grid - acc_est_grid
err_grid = err_grid.reshape(-1, msr_dim)

rel_err = err_grid / ((pot_sim_grid if use_potential else acc_sim_grid) + 1e-20)

# Mean, median, RMS, and SNR of the errors
err_stats = [
    np.mean(err_grid),
    np.median(err_grid),
    np.linalg.norm(err_grid) / np.sqrt(err_grid.size),
]
err_stats.append(np.log10(np.abs(np.mean(1 / (rel_err + 1e-20)))))
print("Error stats: ")
print("Mean: {:.5g} - Median: {:.5g} -  RMS: {:.5g} - SNR (dB):{:.5g}".format(*err_stats))

# 3D plot of the errors
fig_diff =scripts.plot_traj(state_grid, err_grid, val_lbl="Error " + val_lbl)
fig_out_path = os.path.join(out_path, "true_errors_grid.html")
fig_diff.write_html(
        fig_out_path,
        include_mathjax="cdn",
        include_plotlyjs="cdn",
        config=dict({"scrollZoom": True}),
    )
IFrame(src=fig_out_path, width=1000, height=500)

Error stats: 
Mean: -3.2795e-11 - Median: 5.3529e-12 -  RMS: 4.2685e-09 - SNR (dB):8.5645


### Correlation plot

In [13]:
if compute_covariance and n_max<=10:
    # Heatmap of the correlation matrix
    customdata = np.array(
        [
            "{}_{:d},{:d} - {}_{:d},{:d}".format(
                "C" if el_1[0] == 0 else "S",
                el_1[1],
                el_1[2],
                "C" if el_2[0] == 0 else "S",
                el_2[1],
                el_2[2],
            )
            for el_1 in cnm_idx_est
            for el_2 in cnm_idx_est
        ]
    )
    fig_corr = go.Figure(
        data=go.Heatmap(
            z=corr_mat,
            customdata=customdata.reshape(corr_mat.shape),
            hovertemplate="<b>%{customdata}</b><br>%{z:.3f}",
            colorscale="RdBu_r",
        )
    )
    fig_out_path = os.path.join(out_path, "correlations.html")
    fig_corr.write_html(
            fig_out_path,
            include_mathjax="cdn",
            include_plotlyjs="cdn",
            config=dict({"scrollZoom": True}),
        )
    IFrame(src=fig_out_path, width=500, height=500)

### Storing results

In [14]:
# Saving Stokes coefficients in the SHADR format. Can be read via scripts.read_shadr or in SHTOOLS via sh.SHGravCoeffs.from_file
cnm_gt_file = "cnm_ground_truth"
scripts.write_SHADR(
    os.path.join(out_path, cnm_gt_file + ".txt"),
    cnm_vec_sim,
    cnm_map_sim,
    gm=gm,
    r_0=r_0,
)

cnm_sol_file = "cnm_estimated"
scripts.write_SHADR(
    os.path.join(out_path, cnm_sol_file + ".txt"),
    cnm_vec_est,
    cnm_map_est,
    gm=gm,
    r_0=r_0,
)

# Saving measurements evaluated on a spherical grid
grid_gt_file = "grid_ground_truth"
scripts.save_msr_grid(
    os.path.join(out_path, grid_gt_file + ".pkl"),
    rtp_grid,
    pot_sim_grid if use_potential else acc_sim_grid,
)
grid_est_file = "grid_estimated"
scripts.save_msr_grid(
    os.path.join(out_path, grid_est_file + ".pkl"),
    rtp_grid,
    pot_est_grid if use_potential else acc_est_grid,
)